<a href="https://colab.research.google.com/github/Sadiya-Akter-Mim/MovieLens-Recommender-Assignment/blob/main/DataSynthesis_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [5]:
#github raw link

movies_url  = "https://raw.githubusercontent.com/Sadiya-Akter-Mim/MovieLens-Recommender-Assignment/main/movies.csv"
ratings_url = "https://raw.githubusercontent.com/Sadiya-Akter-Mim/MovieLens-Recommender-Assignment/main/ratings.csv"

In [6]:
#load dataset
movies = pd.read_csv(movies_url)
ratings = pd.read_csv(ratings_url)

In [7]:
#show 5 rows
print("Movies dataset:")
print(movies.head())
print("\nRatings dataset:")
print(ratings.head())

Movies dataset:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Ratings dataset:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


load data from github,
print 5 rows to check data,
movie has movieid, title, genres, and
rating has  userid, movieid,ratings,timestamp



In [8]:
# Pivot table: users as rows, movies as columns
rating_matrix = ratings.pivot_table(index="userId", columns="movieId", values="rating")

# Show first rows
print("User-Movie rating matrix:")
print(rating_matrix.head())


User-Movie rating matrix:
movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     NaN     4.0     NaN     NaN     4.0     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           4.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  9       10      ...  193565  193567  193571  193573  193579  193581  \
userId                   ...                                                   
1           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN  ...     NaN     NaN    

Converts ratings into a table where rows = users, columns = movies, here is some empty cells that means user hasnot rated that movie yet,it's need for svd,and collaborative filtering

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
#fill missing rating
rating_filled = rating_matrix.fillna(0)

In [11]:
#compute similarity between users
user_similarity = cosine_similarity(rating_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

print("User similarity matrix:")
print(user_similarity_df.head())

User similarity matrix:
userId       1         2         3         4         5         6         7    \
userId                                                                         
1       1.000000  0.027283  0.059720  0.194395  0.129080  0.128152  0.158744   
2       0.027283  1.000000  0.000000  0.003726  0.016614  0.025333  0.027585   
3       0.059720  0.000000  1.000000  0.002251  0.005020  0.003936  0.000000   
4       0.194395  0.003726  0.002251  1.000000  0.128659  0.088491  0.115120   
5       0.129080  0.016614  0.005020  0.128659  1.000000  0.300349  0.108342   

userId       8         9         10   ...       601       602       603  \
userId                                ...                                 
1       0.136968  0.064263  0.016875  ...  0.080554  0.164455  0.221486   
2       0.027257  0.000000  0.067445  ...  0.202671  0.016866  0.011997   
3       0.004941  0.000000  0.000000  ...  0.005048  0.004892  0.024992   
4       0.062969  0.011361  0.031163  ..

Calculates cosine similarity between users based on their ratings,and this will help to predict which movies user will like



In [12]:
# average ratings
user_mean_ratings = rating_filled.mean(axis=1).values.reshape(-1, 1)

In [13]:
# Predicted ratings = user mean + weighted similarity
predicted_ratings = user_mean_ratings + (user_similarity @ (rating_filled - user_mean_ratings)) / np.abs(user_similarity).sum(axis=1).reshape(-1, 1)

pred_ratings_df = pd.DataFrame(predicted_ratings, index=rating_matrix.index, columns=rating_matrix.columns)

print("Predicted ratings (User-Based CF):")
print(pred_ratings_df.head())

Predicted ratings (User-Based CF):
movieId    1         2         3         4         5         6         7       \
userId                                                                          
1        1.303441  0.558728  0.088670 -0.051682  0.092868  0.552797  0.061363   
2        1.499072  0.711746  0.367174 -0.062537  0.141608  1.058007  0.231937   
3        1.757619  0.737548  0.339010  0.021242  0.252259  0.912959  0.340930   
4        1.708124  0.870158  0.331072  0.019857  0.328812  0.829792  0.385542   
5        1.812384  1.144374  0.630491  0.151292  0.623463  1.114848  0.675453   

movieId    8         9         10      ...    193565    193567    193571  \
userId                                 ...                                 
1       -0.035775 -0.036335  0.562064  ... -0.048683 -0.050523 -0.046844   
2       -0.025217 -0.010468  0.856025  ... -0.085081 -0.085081 -0.085081   
3        0.030206  0.052039  0.919326  ... -0.009822 -0.009907 -0.009737   
4        0.028512

Predict unknown ratings based on similar users, and this help to estimate what movies user might enjoy

In [14]:
from sklearn.decomposition import TruncatedSVD

In [15]:
# Convert filled rating matrix to numpy array
matrix = rating_filled.values

In [16]:
# Apply SVD
svd = TruncatedSVD(n_components=20, random_state=42)
latent_matrix = svd.fit_transform(matrix)
reconstructed = np.dot(latent_matrix, svd.components_)

svd_ratings_df = pd.DataFrame(reconstructed, index=rating_matrix.index, columns=rating_matrix.columns)

print("Predicted ratings (SVD):")
print(svd_ratings_df.head())

Predicted ratings (SVD):
movieId    1         2         3         4         5         6         7       \
userId                                                                          
1        2.789173  1.471277  1.280870 -0.024856  0.254759  1.609531  0.232396   
2        0.016553  0.010828 -0.012655  0.000473 -0.005400 -0.036313 -0.046721   
3       -0.014042  0.047094  0.029815 -0.007728 -0.044938  0.034280 -0.017809   
4        2.564164  0.067424  0.220364  0.101337  0.394431  0.752309  0.489290   
5        1.392494  0.968864  0.353202  0.125885  0.498931  0.649854  0.512006   

movieId    8         9         10      ...    193565    193567    193571  \
userId                                 ...                                 
1        0.115948  0.193227  1.286942  ... -0.009538 -0.008175 -0.010901   
2        0.008091 -0.002748 -0.107562  ...  0.009905  0.008490  0.011320   
3        0.012079 -0.005503  0.016433  ...  0.000003  0.000003  0.000004   
4       -0.036030  0.131220

SVD reduces the large rating matrix into latent features, and reconstructs the matrix to predict missing ratings.It often works better than user-based CF because it captures hidden patterns

In [20]:
def recommend_movies(user_id, N=5, method="svd"):

    #Returns top-N recommended movies for a user.
    #method: 'svd' or 'user'

    if method == "user":
        user_ratings = pred_ratings_df.loc[user_id]
    else:
        user_ratings = svd_ratings_df.loc[user_id]

    # Remove movies already rated
    watched = rating_matrix.loc[user_id].dropna().index
    recommendations = user_ratings.drop(watched).sort_values(ascending=False).head(N)

    return movies[movies["movieId"].isin(recommendations.index)][["movieId", "title"]]

# Example: Top 5 movies for user 1
print("Top 5 recommended movies for user 1:")
display(recommend_movies(1, N=5, method="svd"))


Top 5 recommended movies for user 1:


,movieId,title
507,589,Terminator 2: Judgment Day (1991)
902,1200,Aliens (1986)
958,1259,Stand by Me (1986)
1445,1968,"Breakfast Club, The (1985)"
2078,2762,"Sixth Sense, The (1999)"


Returns the Top-N recommended movies for a given user,removes movies the user already rated, and supports user-based CF (method="user") or SVD (method="svd").